# Data science in Python II. - Classical machine learning methods

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
from tabulate import tabulate

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True

# Scikit-learn, tensorflow, torch, etc.
#import torch
#import tensorflow as tf

from sklearn.datasets import make_regression, make_classification, \
                             make_blobs, make_moons, make_circles
from sklearn.preprocessing import StandardScaler, Normalizer
# ...
# ...

# 2. Training

## 2.1. Regression

<p style="text-align:center; font-size:20px;">
  <b>Data and label -> Model -> Continuous value</b>
</p>

### 2.1.1. Generated datasets

In [ ]:
X, y = make_regression(
    n_samples=5000,
    n_features=10,
    n_informative=10,
    n_targets=1,
    random_state=57
)
X = pd.DataFrame(X)

In [ ]:
X

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5), dpi=120)
ax.grid(True, ls='--', alpha=0.6)

ax.plot(y, lw=2)

ax.set_title("\\textbf{y values}",
             fontsize=30, fontweight='bold')
ax.set_xticks([])

plt.show()

In [ ]:
nr, nc = 2, 5
fig, axes = plt.subplots(nr, nc, figsize=(nc*5, nr*5), dpi=120)

for i, ax in enumerate(axes.flat):
    ax.scatter(X[i], y, 
               color='indianred', alpha=0.6)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel(f'$X_{{{i+1}}}$', fontsize=30, fontweight='bold')
    ax.set_ylabel('$y$', fontsize=30, fontweight='bold')

plt.show()

### Pull up the regression methods...

There's lots of them...: https://en.wikipedia.org/wiki/Outline_of_machine_learning#Regression_analysis

#### 1. Split the dataset to a train and a test set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.33,
    random_state=57
)

In [ ]:
nr, nc = 2, 5
fig, axes = plt.subplots(nr, nc, figsize=(nc*5, nr*5), dpi=120)

for i, ax in enumerate(axes.flat):
    ax.scatter(X_train[i], y_train, label='Train set',
               color='indianred', s=4**2, alpha=0.6)
    ax.scatter(X_test[i], y_test, label='Test set',
               color='cornflowerblue', s=4**2, alpha=0.6)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel(f'$X_{{{i+1}}}$', fontsize=30, fontweight='bold')
    ax.set_ylabel('$y$', fontsize=30, fontweight='bold')
    ax.legend(loc='lower right', fontsize=15)

plt.show()

#### 2. Train and evaluate some linear models

In [ ]:
from sklearn.linear_model import ARDRegression, BayesianRidge, ElasticNet, \
                                 Lasso, LinearRegression, Ridge, SGDRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score

In [ ]:
def regression(model, *, X_train, y_train, X_test, y_test):
    reg = model()
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    print(f"Score for {model.__name__} : {r2_score(y_test, y_pred):.4f}")

In [ ]:
models = [
    ARDRegression, BayesianRidge, ElasticNet,
    Lasso, LinearRegression, Ridge, SGDRegressor, SVR
]
for model in models:
    regression(model,
               X_train=X_train, y_train=y_train,
               X_test=X_test, y_test=y_test)

### Fun fact: "Regression" and "linear regression" are not necessarily "linear"

In [ ]:
# Generate an oddly specific toy dataset, which 99% of the times are
# shown as an example, when this fun fact arises.
X = np.linspace(-40, 40, 500)
y_sq = 1/4 * X**2
y_li = -3.4 * X + 2
y_tr = 90 * np.cos(X)**3
y = y_sq + y_li + y_tr

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6),
                       facecolor='black', subplot_kw={'facecolor' : 'black'})
ax.grid(True, ls='--', color='.7', alpha=0.4)
ax.scatter(X, y, label='Original data',
           color=cm.magma(y/y.max()/2 + 0.5), ec='none', s=7**2, alpha=0.7)
ax.tick_params(axis='both', which='major', labelsize=16, colors='white')

plt.show()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer

Using `sklearn` we can sequentially transform a dataset and then fit an estimator on it. This sequential list of transformations, finished by a single, final estimator are referred to as a "pipeline" in `sklearn`. The pipeline below defined as

```python
pipeline = Pipeline([("polynomial_variation", FunctionTransformer(poly2_reg)),
                     ("linear_regression", LinearRegression())])
```

contains some arbitrary transformation defined by the `poly2_reg` function (this can be anything actually that transform `X` in any way), which is then fitted using `sklearn`'s built-in linear estimator, the `LinearRegressor`, which implements the ordinary least squares linear regression.

In [ ]:
def poly2_reg(X):
    """
    Returns the transformed array using the equation
       ```A * X^2 + B * cos^3(X) + C * X + D```
    """
    return np.hstack((np.cos(X)**3, X, X**2))

In [ ]:
pipeline = Pipeline([("polynomial_variation", FunctionTransformer(poly2_reg)),
                     ("linear_regression", LinearRegression())])
# Transform X for the PolynomialFeatures() and LinearRegression() class
# Then fit on the pipeline the available data
pipeline.fit(X[:, np.newaxis], y)
# Get coefficients
c, b, a = pipeline[1].coef_
d = pipeline[1].intercept_

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6),
                       facecolor='black', subplot_kw={'facecolor' : 'black'})
ax.grid(True, ls='--', color='.7', alpha=0.4)
ax.scatter(X, y, label='Original data',
             color=cm.magma(y/y.max()/2 + 0.5), ec='none', s=7**2, alpha=0.7)
ax.plot(X, pipeline.predict(X[:, np.newaxis]), label='Fitted model',
        color='tab:green', lw=4, ls='--', alpha=0.8)

title = f'Equation of fitted polynomial: \n' + \
        f'$({a:.3f}\,x^2) + ({b:.3f}\,x) + ({c:.3f}\,\cos^3(x)) + ({d:.3f})$'
ax.set_title(title, fontsize=16, fontweight='bold', color='white')

ax.tick_params(axis='both', which='major', labelsize=16, colors='white')

ax.legend(loc='best', fontsize=24,
          facecolor='black', labelcolor='white')

plt.show()

## 2.1.2. Use of some real data finally...

Here we're using the [Communities and Crime Data Set](https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime) from UCI, which contains detailed crime statistics from the US. Using `LinearRegression` and the `Lasso` regression models, we're trying to find which feature attributes the most to the crime rate in the US?

In [ ]:
from urllib import request

In [ ]:
features = []
# Feature names start with `@attribute`, followed by the feature name,
# then ending with the type of the feature values (numeric/string/etc.)
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.names'
with request.urlopen(url) as f:
    while True:
        line = f.readline().decode("utf-8")
        if not line:
            break
        if '@attribute' in line:
            features.append(line.strip().split(' ')[1])

In [ ]:
# Missing values are marked with an `?` in the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data'
df = pd.read_csv(url, sep=',', names=features, na_values=['?'])

In [ ]:
display(df.head())
display(df.tail())

### 2.1.2./a. Preprocess dataset

#### Handle missing/ID labels

While missing values in meaningful features should be filled appropriately, columns representing ID-like variables can be deleted. Location and violent crime rates do correlates in real life, but an idea of a causal relationship between location and crime rates can be discarded now. Description for each feature can be accessed in the `community.names` file.

#### ID-like columns
The first 4 columns (`state`, `county`, `community`, `communityname`) can be deleted, because they represent locational data.

In [ ]:
df = df[features[4:]]

The column `fold` is a debug feature from cross-validation, which can be also discarded for now.

In [ ]:
df = df[features[5:]]

#### Features with missing values

According to the feature descriptions, all remaining columns are in a decimal format and scaled into the interval of $\left[ 0, 1 \right]$. The only exception is the feature `LemasGangUnitDeploy`, which is actually an ordinal with values $0.0$, $0.5$ and $1.0$. We can still however handle it as a decimal feature.

There is a table in the `community.names` description file, which summarize the basic statistical attributes (mean, median, standard deviation, etc.) of each features in the dataset. According to this table any features with missing entries have exactly $1675$ missing values in each of them. (There is only one exception, the column `OtherPerCap`, where only $1$ value is missing.). It is entirely logical to assume that in this case the missing features are always missing from the same lines. If the hypothesis is true, we can test it by visualizing the missing values on a matrixplot. If we plot features on the $y$-axis, we should see only horizontal lines (which are interrupted by vertical gaps) in the dataset, instead of individual points scattered all around the dataset.

In [ ]:
figsize = 100
fig, axes = plt.subplots(figsize=(figsize,figsize))
axes.set_aspect('equal')
axes.grid(False)

axes.imshow(df.isna().T, interpolation='none')
axes.set_xlabel('Locations', fontsize=50, fontweight='bold')
axes.set_ylabel('Features', fontsize=50, fontweight='bold')

plt.show()

These are indeed "horizontal lines interrupted by vertical gaps". However these features miss most of their values. In this case we should consider simply dropping these features from the model, since filling them up with artificial values could reasonably distort the impact of these features on our model. I'll try this method for this dataset.

In [ ]:
# Drop columns with at least 50% of values missing
df_n = df.dropna(axis=1, thresh=int(0.5 * len(df)), inplace=False)

# Fill that 1 remaining entry with the mean of the corresponding feature
df_n = df_n.fillna(df_n.mean())

In [ ]:
figsize = 100
fig, axes = plt.subplots(figsize=(figsize,figsize))
axes.set_aspect('equal')
axes.grid(False)

axes.imshow(df_n.isna().T)
axes.set_xlabel('Locations', fontsize=50, fontweight='bold')
axes.set_ylabel('Features', fontsize=50, fontweight='bold')

plt.show()

#### Scale dataset

In [ ]:
# Create the X and y datasets
X = df_n[df_n.columns[:-1]]
y = df_n[df_n.columns[-1]]
# Scale the dataset
X = StandardScaler().fit_transform(X)

### 2.1.2./b. Fit linear regression using 5-fold CV

<img width="800px" src="./images/5foldcv.png" style="display:block; margin:auto;"/>

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
# Number of folds
folds = 5
# Invoke the KFold class from sklearn for CV tests
cv = KFold(n_splits=folds, shuffle=True, random_state=42)
# The model we use is linear regression
model = LinearRegression()

In [ ]:
# Test R^2 score
# Refrence: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
scores = cross_val_score(model, X, y, scoring='r2', cv=cv)

print('KFOLD SCORES:\n' +
      '----------------')
print(scores)
print('Mean of scores : {0:.4f}'.format(np.mean(scores)))
print('Std of scores : {0:.4f}'.format(np.std(scores)))

### 2.1.2./d. Fit Lasso regression using 5-fold CV

In [ ]:
from sklearn.linear_model import Lasso

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
# Use just part of the full dataset for training with 5-fold CV
# Use the remaining values as a test dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [ ]:
# 5-fold search is needed
folds = 5
cv = KFold(n_splits=folds, shuffle=True, random_state=42)
# Lasso estimator
model = Lasso(random_state=None)
# Paramters to explored:
# alpha, normalize, max_iter
param_grid = {
    'alpha' : np.logspace(-10, 0.1, 20),
    'max_iter' : np.logspace(4, 6, 10, dtype=int)
}
# Grid search cross-validation
clf = GridSearchCV(estimator=model,
                   param_grid=param_grid,
                   cv=cv,
                   n_jobs=-1)

In [ ]:
best_model = clf.fit(X_train, y_train).best_estimator_

In [ ]:
print('Best model : {0}'.format(best_model))
y_pred = best_model.predict(X_test)
print(f"Score for Lasso : {r2_score(y_test, y_pred):.4f}")

In [ ]:
fig, axes = plt.subplots(figsize=(5, 5), dpi=120,
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
axes.set_aspect('equal')

axes.plot([0, 1], [0, 1],
          color=cm.magma(0.93), lw=4, ls='--', zorder=3, alpha=0.5)
axes.scatter(y_test, y_pred,
             color=cm.magma(0.5), s=12**2, ec='black', alpha=0.4)

axes.set_xlim(0, 1)
axes.set_ylim(0, 1)

axes.set_title(f'R$^{2}$ score : {r2_score(y_test, y_pred):.4f}',
               fontsize=16, fontweight='bold', color='white')
axes.set_xlabel('$\mathrm{y_{groundtruth}}$',
                fontsize=30, fontweight='bold', color='white')
axes.set_ylabel('$\mathrm{y_{pred}}$',
                fontsize=30, fontweight='bold', color='white')
axes.tick_params(axis='both', which='major',
                 labelsize=20, colors='white', rotation=20)

fig.suptitle('Predictions made with the optimized\n5-fold Lasso regression.',
             color='white', fontsize=21, y=-0.1)

plt.show()

#### Using the models as above

In [ ]:
models = [
    ARDRegression, BayesianRidge, ElasticNet,
    Lasso, LinearRegression, Ridge, SGDRegressor, SVR
]
for model in models:
    regression(model,
               X_train=X_train, y_train=y_train,
               X_test=X_test, y_test=y_test)

### Notes on the results in 2.1.2./d.

The grid search returned a very small (almost the smallest) alpha value in the analysis above. This wasn't actually an error, but the indication, that a linear regression could be efficiently used in case of this specific dataset. ($\alpha \to 0$ is equivalent to the linear regression in the case of the Lasso regression.)

### 2.1.2./e. Lasso evaluation with shrinkage method

In [ ]:
def evaluate_lasso(X, y, alpha=1.0, normalize=False, max_iter=1e5):

    model = Lasso(alpha=alpha, max_iter=max_iter, random_state=None)
    model.fit(X, y)

    return model

In [ ]:
lasso_alphas = []
lasso_coeffs = []

for a in np.logspace(-10, 1, 100):
    lasso_alphas.append(a)
    model = evaluate_lasso(X_train, y_train, alpha=a, normalize=False, max_iter=10000)
    lasso_coeffs.append(model.coef_)

lasso_alphas = np.array(lasso_alphas)
lasso_coeffs = np.array(lasso_coeffs)

In [ ]:
nr, nc = 1, 2
fig, axes = plt.subplots(nr, nc, figsize=(nc*10, nr*10),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})

ax = axes[0]
ax.set_xlim(np.log(lasso_alphas.min()), np.log(lasso_alphas.max()))
ax.set_title('Full test range',
             fontsize=16, fontweight='bold', color='white')

ax = axes[1]
ax.set_xlim(-6, -1)
ax.set_ylim(-0.08, 0.08)
ax.set_title('Zoomed on interesting area',
             fontsize=16, fontweight='bold', color='white')

for ax in axes:
    ax.plot(np.log(lasso_alphas), lasso_coeffs,
          lw=3, alpha=0.6)

    ax.set_xlabel('$\log \\left( \\alpha \\right)$',
                  fontsize=16, fontweight='bold', color='white')
    ax.set_ylabel('Value of coefficients',
                  fontsize=16, fontweight='bold', color='white')
    ax.tick_params(axis='both', which='major',
                   labelsize=12, colors='white', rotation=20)

fig.suptitle('Shrinkage method used on the results of Lasso regression.',
             color='white', fontsize=21, y=0.03)

plt.show()

Around $\log \left( \alpha \right)\approx-5$ is where mostly the interesting events happen. That's the place, where a lot of coefficients diverges away from 0, while other coefficients vanish. Two other coefficients does the same, but with much a much smaller extent around $\log \left( \alpha \right) \approx -12$, before vanishing quickly. Let's see which features are responsible for this last anomalies.

In [ ]:
fig, ax = plt.subplots(figsize=(35, 35))

ax.imshow(lasso_coeffs.T, aspect=0.8, cmap='magma')

ax.set_xticks([])
ax.set_xticklabels([])

ax.set_yticks([i for i in range(len(df_n.columns[:-1]))])
ax.set_yticklabels(df_n.columns.tolist()[:-1])

plt.show()

## 2.2. Clustering (#3 in the `problems.ipynb`)

In [ ]:
N = 1500
# Create a dummy dataset of blobs
Xb, yb = make_blobs(
    n_samples=N,    # Number of points in the dataset
    n_features=2,   # Dimension of the dataset (Here it's a 2D dataset)
    centers=3,      # Number of blobs to create
    cluster_std=[1.0, 2.5, 0.5],
    center_box=(-10, 10),
    random_state=57
)

# Create a dummy dataset of circles
Xc, yc = make_circles(
    n_samples=N,    # Number of points in the dataset
    noise=0.05,
    factor=0.5,
    random_state=57
)

# Create a dummy dataset of moons
Xm, ym = make_moons(
    n_samples=N,    # Number of points in the dataset
    noise=0.1,
    random_state=57
)

In [ ]:
# Visualize them
nr, nc = 1, 3
fig, axes = plt.subplots(nrows=nr, ncols=nc, figsize=(8*nc, 8*nr))

Xi = (Xb, Xc, Xm)
yi = (yb, yc, ym)
for X, y, ax in zip(Xi, yi, axes.flat):
    ax.grid(True, ls='--', alpha=0.6)

    X = X - np.mean(X)
    ax.scatter(*X.T, c=y)

    lim = 1.1 * np.max(np.abs(X))
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)

plt.show()

### An example for clustering: naive *k*-means algorithm

<img src="./images/kmeans.gif" style="display:block; margin:auto;"/>

### Compare different types of clustering methods

In [ ]:
import time

from itertools import cycle, islice
from sklearn import cluster, mixture
from sklearn.cluster import MeanShift, MiniBatchKMeans, AffinityPropagation, \
                            AgglomerativeClustering, SpectralClustering, \
                            DBSCAN, OPTICS, Birch
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import kneighbors_graph

### Define the datasets and corresponding clustering parameters

In [ ]:
datasets = [
  (
    Xb, {}
  ),
  (
    Xc, {
      "damping": 0.77,
      "preference": -240,
      "quantile": 0.2,
      "min_samples": 20,
      "xi": 0.25,
      "n_clusters": 2,
    }
  ),
  (
    Xm, {
      "damping": 0.75,
      "preference": -220,
      "n_clusters": 2
    }
  )
]

default_params = {
    "quantile": 0.3,
    "eps": 0.3,
    "damping": 0.9,
    "preference": -200,
    "n_neighbors": 10,
    "n_clusters": 3,
    "min_samples": 20,
    "xi": 0.05,
    "min_cluster_size": 0.1,
}

### Define different clustering methods

In [ ]:
def return_clustering_algos(params, **kwargs):

    bandwidth = None if not kwargs else kwargs["bandwidth"]
    connectivity = None if not kwargs else kwargs["connectivity"]

    ms = MeanShift(
        bandwidth=bandwidth,
        bin_seeding=True
    )
    two_means = MiniBatchKMeans(
        n_clusters=params["n_clusters"]
    )
    affinity_propagation = AffinityPropagation(
        damping=params["damping"],
        preference=params["preference"],
        random_state=0
    )
    ward = AgglomerativeClustering(
        n_clusters=params["n_clusters"],
        linkage="ward",
        connectivity=connectivity
    )
    average_linkage = AgglomerativeClustering(
        linkage="average",
        metric="cityblock",
        n_clusters=params["n_clusters"],
        connectivity=connectivity,
    )
    spectral = SpectralClustering(
        n_clusters=params["n_clusters"],
        eigen_solver="arpack",
        affinity="nearest_neighbors",
    )
    dbscan = DBSCAN(
        eps=params["eps"]
    )
    optics = OPTICS(
        min_samples=params["min_samples"],
        xi=params["xi"],
        min_cluster_size=params["min_cluster_size"],
    )
    birch = Birch(
        n_clusters=params["n_clusters"]
    )
    gmm = GaussianMixture(
        n_components=params["n_clusters"],
        covariance_type="full"
    )

    clustering_algorithms = (
        ("MeanShift", ms),
        ("MiniBatch\nKMeans", two_means),
        ("Affinity\nPropagation", affinity_propagation),
        ("Ward", ward),
        ("Agglomerative\nClustering", average_linkage),
        ("Spectral\nClustering", spectral),
        ("DBSCAN", dbscan),
        ("OPTICS", optics),
        ("BIRCH", birch),
        ("Gaussian\nMixture", gmm),
    )

    return clustering_algorithms

#### Stolen and reworked from matplotlib's website

In [ ]:
# ============
# Set up cluster parameters
# ============
nr, nc = len(datasets), len(return_clustering_algos(default_params))
fig, axes = plt.subplots(nr, nc, figsize=(4*nc, 4*nr))
fig.subplots_adjust(
    left=0.02, right=0.98, bottom=0.001, top=0.95, wspace=0.05, hspace=0.01
)


for dataset_i, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_params.copy()
    params.update(algo_params)

    #X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(dataset)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params["quantile"])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params["n_neighbors"], include_self=False
    )
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    extra_params = {"bandwidth" : bandwidth, "connectivity" : connectivity}

    # ============
    # Create cluster objects
    # ============
    clustering_algorithms = return_clustering_algos(params, **extra_params)

    for ax_i, (name, algorithm) in enumerate(clustering_algorithms):
        # Fit
        t0 = time.time()
        algorithm.fit(X)
        t1 = time.time()
        if hasattr(algorithm, "labels_"):
            y_pred = algorithm.labels_.astype(int)
        else:
            y_pred = algorithm.predict(X)

        ax = axes[dataset_i, ax_i]
        ax.axis('off')
        if dataset_i == 0:
            ax.set_title(name, fontsize=18)

        colors = np.array(
            list(
                islice(
                    cycle(
                        [
                            "#377eb8",
                            "#ff7f00",
                            "#4daf4a",
                            "#f781bf",
                            "#a65628",
                            "#984ea3",
                            "#999999",
                            "#e41a1c",
                            "#dede00",
                        ]
                    ),
                    int(max(y_pred) + 1),
                )
            )
        )
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        ax.scatter(*X.T, color=np.array(colors)[y_pred], s=16)

        lim = 1.1 * np.max(np.abs(X))
        ax.set_xlim(-lim, lim)
        ax.set_ylim(-lim, lim)

plt.show()

## 3. Classification (#2 in the `problems.ipynb`)

In [ ]:
X, y = make_classification(
    n_samples=100,    # Number of points in the data set
    n_features=6,     # Number of features in the data set
    n_informative=4,
    n_redundant=2,
    n_repeated=0,
    n_classes=2,
    n_clusters_per_class=2,
    random_state=0,
)

In [ ]:
pd.DataFrame(X)

### Literally the same as regression, but with different methods...